Import All Data Files

In [14]:
# import packages
#!pip install geopy
import pandas as pd
import sklearn.neighbors as NN
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import datetime
import re
import sklearn.metrics as metrics
import geopy.distance
import seaborn as sns
from matplotlib.colors import ListedColormap

data_per_file = 1/13

# import data
data = pd.DataFrame()
lens = []
for m in range(1, 13):
    path = None # Modify the 3 paths below based on where the data folder is located in your filesystem
    if m <= 5:
        path = r"C:\Users\there\CS-396-Divvy-Bikes\Data\20210"+str(m)+"-divvy-tripdata.csv"
    if (m >= 5) & (m < 10):
        path = r"C:\Users\there\CS-396-Divvy-Bikes\Data\20200"+str(m)+"-divvy-tripdata.csv"
    elif m >= 10:
        path = r"C:\Users\there\CS-396-Divvy-Bikes\Data\2020"+str(m)+"-divvy-tripdata.csv"
    input_data = pd.read_csv(path)
    split_d = input_data.iloc[:int(len(input_data)*data_per_file)]
    data = data.append(split_d, ignore_index=True)
print(data)

                 ride_id  rideable_type        started_at          ended_at  \
0       B9F73448DFBE0D45   classic_bike   1/24/2021 19:15   1/24/2021 19:22   
1       457C7F4B5D3DA135  electric_bike   1/23/2021 12:57   1/23/2021 13:02   
2       57C750326F9FDABE  electric_bike    1/9/2021 15:28    1/9/2021 15:37   
3       4D518C65E338D070  electric_bike    1/9/2021 15:28    1/9/2021 15:37   
4       9D08A3AFF410474D   classic_bike   1/24/2021 15:56   1/24/2021 16:07   
...                  ...            ...               ...               ...   
226175  9D70D78AEF553C46  electric_bike   12/9/2020 18:02   12/9/2020 18:48   
226176  9DC34FD8AB4AAF56   classic_bike  12/22/2020 23:59   12/23/2020 0:07   
226177  A467B367F2F3923C    docked_bike   12/2/2020 12:36   12/2/2020 12:45   
226178  1654D4F3BE89E875   classic_bike   12/9/2020 15:14   12/9/2020 15:24   
226179  864B573E1A1DFA6B   classic_bike  12/13/2020 10:59  12/13/2020 11:23   

                start_station_name start_station_id

In [15]:
data.to_csv(r"C:\Users\there\CS-396-Divvy-Bikes\Data\all-months-divvy-tripdata.csv")